In [23]:
import psycopg2
from password import password

In [24]:
password = password

In [25]:
def connect_and_make_cursor(host, dbname, user, password):
    """host = IP of host service
    dbname = database which to connect to
    user = username
    password = password associated with username"""
    
    try:
        conn = psycopg2.connect(f"host = {host}, dbname = {dbname} user = {user} password = {password}")
        conn.set_session(autocommit = True)
        cur = conn.cursor()
    except psycopg2.Error as e:
        print(e)
    return conn, cur

In [26]:
conn, cur = connect_and_make_cursor('127.0.0.1', 'postgres', 'postgres', password)

# CREATE DATABASE

In [27]:
cur.execute("CREATE DATABASE acme_co")

In [28]:
def close_connection():
    """closes both cursor and connection in that order"""
    
    cur.close()
    conn.close()

In [29]:
close_connection()

In [30]:
conn, cur = connect_and_make_cursor('127.0.0.1', 'acme_co', 'postgres', password)

# CREATE TABLES

** Without functions **

In [31]:
#try:
#    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
#except psycopg2.Error as e:
#        print(e)

In [32]:
#try:
#    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
#except psycopg2.Error as e:
#    print(e)

In [33]:
#try:
#    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
#except psycopg2.Error as e:
#   print(e)

In [34]:
#try:
#    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
#except psycopg2.Error as e:
#   print(e)

In [35]:
#try:
#    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
#except psycopg2.Error as e:
#    print(e)

In [36]:
#try:
#    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
#except psycopg2.Error as e:
#    print(e)

** Using Function **

In [37]:
def create_table(table_name):
    """table_name = name of the table which you like to create"""
    try:
        cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ();")
    except psycopg2.Error as e:
        print(e)

In [38]:
tables = ['employee', 'sales_dept', 'it_dept', 'customers', 'inventory', 'sales']

for table in tables:
    create_table(table)

In [39]:
def insert_columns(table_name, col_type_dict):
    """ table_name = Name of Table to insert columns
        col_type_dict = Dictionary containing key value pairs of column name : column type"""
    try:
        for col_name, type_ in col_type_dict.items():
            cur.execute(f"ALTER TABLE {table_name} \
                        ADD COLUMN IF NOT EXISTS {col_name} {type_};")
    except psycopg2.Error as e:
        print(e)

In [40]:
employee_cols = {'emp_id' : 'SERIAL PRIMARY KEY', 'first_name': 'varchar', 'last_name': 'varchar', 'position': 'varchar', 'start_date': 'DATE DEFAULT CURRENT_DATE', 'phone': 'varchar', 'email': 'varchar'}

In [41]:
insert_columns('employee', employee_cols)

In [42]:
def get_column_info(table_name):
    """returns column names of a specific table"""
    
    try:
        cur.execute(f"select column_name, data_type, character_maximum_length \
        from INFORMATION_SCHEMA.COLUMNS where table_name = '{table_name}';")
    except psycopg2.Error as e:
        print(e)
        
    return cur.fetchall()

In [43]:
get_column_info('employee')

[('emp_id', 'integer', None),
 ('first_name', 'character varying', None),
 ('last_name', 'character varying', None),
 ('position', 'character varying', None),
 ('start_date', 'date', None),
 ('phone', 'character varying', None),
 ('email', 'character varying', None)]

** Now that our tables are created and columns defined lets look at inserting Rows without functions **

In [279]:
try:
    cur.execute("INSERT INTO table_name (id, name, contact, position) \
    VALUES (%s, %s, %s, %s)", (1, 'bob' '123-456-7890', 'developer'))
except psycopg2.Error as e:
    print(e)

SyntaxError: invalid syntax (<ipython-input-279-ba156924a0d3>, line 2)

In [280]:
try:
    cur.execute("INSERT INTO table_name (id, name, contact, position) \
    VALUES (%s, %s, %s, %s)", (1, 'james' 123-456-7890, 'developer'))
except psycopg2.Error as e:
    print(e)

SyntaxError: invalid syntax (<ipython-input-280-ba156924a0d3>, line 2)

In [281]:
try:
    cur.execute("INSERT INTO table_name (id, name, contact, position) \
    VALUES (%s, %s, %s, %s)", (1, 'bob' 123-456-7890, 'developer'))
except psycopg2.Error as e:
    print(e)

SyntaxError: invalid syntax (<ipython-input-281-ba156924a0d3>, line 2)

In [282]:
try:
    cur.execute("INSERT INTO table_name (id, name, contact, position) \
    VALUES (%s, %s, %s, %s)", (1, 'bob' 123-456-7890, 'developer'))
except psycopg2.Error as e:
    print(e)

SyntaxError: invalid syntax (<ipython-input-282-ba156924a0d3>, line 2)

In [283]:
try:
    cur.execute("INSERT INTO table_name (id, name, contact, position) \
    VALUES (%s, %s, %s, %s)", (1, 'bob' 123-456-7890, 'developer'))
except psycopg2.Error as e:
    print(e)

SyntaxError: invalid syntax (<ipython-input-283-ba156924a0d3>, line 2)

** Functional Row Insertion **

In [45]:
def insert_rows(table_name, cols, data):
    """table_name = table which to insert rows into
    cols = list of columns which will recieve the data. Expects string in format "(col1, col2, col3, coln)"
    data = the actual data which you will like to insert into each row contained in tuple"""
    
    count = len(data)
    query = f"INSERT INTO {table_name} {cols} \
                    VALUES (" + "%s,"*count  + ")"
    query = query[::-1].replace(',', '', 1)
    query = query[::-1]
    try:
        cur.execute(query, data)
    except psycopg2.Error as e:
        print(e)

In [46]:
employee_list = [('Bob', 'Smith', 'President', '510-123-4567', 'bob_smith@acme.com'),
                ('Jane', 'Smith', 'Vice President', '510-123-4537', 'jane_smith@acme.com'),
                ('Tom', 'Harper', 'Developer', '510-123-4577', 'tom_h@acme.com'),
                ('Vina', 'Nguyen', 'Sales', '510-123-4517', 'vina_n@acme.com'),
                ('Lee', 'Garr', 'Sales', '510-123-4597', 'lee_g@acme.com')]

for employee in employee_list:
    insert_rows('employee', "(first_name, last_name, position, phone, email)", employee)

In [47]:
cur.execute("SELECT * FROM employee")
cur.fetchall()

[(1,
  'Bob',
  'Smith',
  'President',
  datetime.date(2019, 4, 2),
  '510-123-4567',
  'bob_smith@acme.com'),
 (2,
  'Jane',
  'Smith',
  'Vice President',
  datetime.date(2019, 4, 2),
  '510-123-4537',
  'jane_smith@acme.com'),
 (3,
  'Tom',
  'Harper',
  'Developer',
  datetime.date(2019, 4, 2),
  '510-123-4577',
  'tom_h@acme.com'),
 (4,
  'Vina',
  'Nguyen',
  'Sales',
  datetime.date(2019, 4, 2),
  '510-123-4517',
  'vina_n@acme.com'),
 (5,
  'Lee',
  'Garr',
  'Sales',
  datetime.date(2019, 4, 2),
  '510-123-4597',
  'lee_g@acme.com')]

# Whoops. The start_date is wrong for our President and Vice President. Let's update these values

In [49]:
def update_row(table_name, col_name, data, condition = None):
    """table_name = table which to update
    col_name = column in which to update the value
    data = data which you would like to change the field to.
    condition = Optional. Conditional requirement with which to query using WHERE clause"""
    
    if condition != None:
        try:
            cur.execute(f"UPDATE {table_name}\
            SET {col_name} = {data} \
            WHERE {condition};")
        except psycopg2.Error as e:
            print(e)
            
    else:
        try:
            cur.execute(f"UPDATE {table_name}\
            SET {col_name} = {data};")
        except psycopg2.Error as e:
            print(e)
        

In [50]:
update_row('employee', 'start_date', "'2019/01/23'", "LAST_NAME = 'Smith'")

In [51]:
cur.execute("SELECT * FROM employee")
cur.fetchall()

[(3,
  'Tom',
  'Harper',
  'Developer',
  datetime.date(2019, 4, 2),
  '510-123-4577',
  'tom_h@acme.com'),
 (4,
  'Vina',
  'Nguyen',
  'Sales',
  datetime.date(2019, 4, 2),
  '510-123-4517',
  'vina_n@acme.com'),
 (5,
  'Lee',
  'Garr',
  'Sales',
  datetime.date(2019, 4, 2),
  '510-123-4597',
  'lee_g@acme.com'),
 (1,
  'Bob',
  'Smith',
  'President',
  datetime.date(2019, 1, 23),
  '510-123-4567',
  'bob_smith@acme.com'),
 (2,
  'Jane',
  'Smith',
  'Vice President',
  datetime.date(2019, 1, 23),
  '510-123-4537',
  'jane_smith@acme.com')]

# Extract sales staff data in order to create our sales_dept table

In [290]:
cur.execute("SELECT emp_id FROM employee \
WHERE position = 'Sales'")

sales_staff = cur.fetchall()
sales_staff

[(4,), (5,)]

In [55]:
sales_cols = {'emp_id' : 'int PRIMARY KEY', 'total_sales': 'int', 'region': 'varchar'}

In [56]:
insert_columns('sales_dept', sales_cols)

In [61]:
get_column_info('sales_dept')

[('emp_id', 'integer', None),
 ('total_sales', 'integer', None),
 ('region', 'character varying', None)]

In [58]:
sales_list = [(4, 10000, 'NW'), (5, 5000, 'SW')]

for sales_rep in sales_list:
    insert_rows('sales_dept', "(emp_id, total_sales, region)", sales_rep)

In [59]:
cur.execute("SELECT * FROM sales_dept")
cur.fetchall()

[(4, 10000, 'NW'), (5, 5000, 'SW')]

** Fetch employee name and total sales using inner join on employee and sales department **

In [62]:
cur.execute("SELECT EMPLOYEE.first_name, EMPLOYEE.last_name, SALES_DEPT.total_sales \
FROM employee \
INNER JOIN sales_dept ON sales_dept.emp_id = employee.emp_id")

In [63]:
cur.fetchall()

[('Vina', 'Nguyen', 10000), ('Lee', 'Garr', 5000)]

# Create IT dept table

In [65]:
get_column_info('it_dept')

[]

In [66]:
cur.execute("SELECT * FROM employee WHERE position = 'Developer'")
cur.fetchall()

[(3,
  'Tom',
  'Harper',
  'Developer',
  datetime.date(2019, 4, 2),
  '510-123-4577',
  'tom_h@acme.com')]

** Skills will contain several different values so we will want to create it as an array **

In [67]:
it_cols = {'emp_id' : 'int PRIMARY KEY', 'skills': 'text[]', 'remote': 'bool'}

insert_columns('it_dept', it_cols)

In [69]:
get_column_info('it_dept')

[('emp_id', 'integer', None),
 ('skills', 'ARRAY', None),
 ('remote', 'boolean', None)]

In [70]:
insert_rows('it_dept', "(emp_id, skills, remote)", (3, ['Python', 'SQL', 'Javascript'], True))

In [71]:
cur.execute("SELECT * FROM it_dept")
cur.fetchall()

[(3, ['Python', 'SQL', 'Javascript'], True)]

** Query by individual programming skill set **

In [72]:
cur.execute("SELECT * FROM it_dept WHERE 'Python' = ANY(skills)")
cur.fetchall()

[(3, ['Python', 'SQL', 'Javascript'], True)]

# Create customers Table

In [76]:
cust_columns = {"cust_id": "SERIAL PRIMARY KEY", "first_name": 'varchar', 'last_name': 'varchar', 'sales_man': 'int references sales_dept(emp_id)', 'contact': 'varchar'}

insert_columns('customers', cust_columns)

In [77]:
customers = [('janet', 'dane', 4, 'janet_dane@gmail.com'), ('Joey', 'Capone', 5, 'joey_capone@gmail.com')]

In [78]:
for customer in customers:
    insert_rows('customers', "(first_name, last_name, sales_man, contact)", customer)

In [79]:
cur.execute("SELECT * FROM customers")
cur.fetchall()

[(1, 'janet', 'dane', 4, 'janet_dane@gmail.com'),
 (2, 'Joey', 'Capone', 5, 'joey_capone@gmail.com')]

# CREATE SALES TABLE

In [90]:
sales_cols = {'order_id' : 'SERIAL PRIMARY KEY', 'cust_id': 'int references customers(cust_id)', 'purchase_total': 'int',
             'total_items': 'int', 'salesman_id': 'int references sales_dept(emp_id)', 'description': "text[]" }

In [91]:
insert_columns('sales', sales_cols)

multiple primary keys for table "sales" are not allowed



In [92]:
dummy_sales = [(1, 600, 2, 4, ['GPU', 'monitor']), (2, 1200, 3, 5, ['GPU', 'monitor', 'CPU'])]

for sale in dummy_sales:
    insert_rows('sales', "(cust_id, purchase_total, total_items, salesman_id, description)", sale)

column "cust_id" of relation "sales" does not exist
LINE 1: INSERT INTO sales (cust_id, purchase_total, total_items, sal...
                           ^

column "cust_id" of relation "sales" does not exist
LINE 1: INSERT INTO sales (cust_id, purchase_total, total_items, sal...
                           ^



In [93]:
get_column_info('sales')

[('emp_id', 'integer', None),
 ('total_sales', 'integer', None),
 ('region', 'character varying', None)]

In [88]:
cur.execute("SELECT * FROM sales")
cur.fetchall()

[]

** Let's query the average purchase total and round to two decimal places**

In [314]:
cur.execute("SELECT ROUND(AVG(purchase_total), 2) FROM sales;")
cur.fetchall()

[(Decimal('900.00'),)]

** Now we will query all purchase descriptions sold, salesman first and last name associated with purchase and the region from which they work. This will require 2 joins.**

In [315]:
cur.execute("SELECT sales.description, sales.total_items, sales_dept.region, \
employee.first_name, employee.last_name FROM SALES \
INNER JOIN sales_dept ON sales.salesman_id = sales_dept.emp_id \
INNER JOIN employee ON sales_dept.emp_id = employee.emp_id")
cur.fetchall()

[(['GPU', 'monitor'], 2, 'NW', 'Vina', 'Nguyen'),
 (['GPU', 'monitor', 'CPU'], 3, 'SW', 'Lee', 'Garr')]

# Create Inventory Table

In [316]:
inventory_cols = {"item_id": "SERIAL PRIMARY KEY", "item_name": "varchar", "in_stock": "bool", "total": "int"}

In [317]:
insert_columns("inventory", inventory_cols)

In [318]:
get_column_names('inventory')

[('item_id', 'integer', None),
 ('item_name', 'character varying', None),
 ('in_stock', 'boolean', None),
 ('total', 'integer', None)]

** Whoops forgot to add our wholesale price and retail price columns in the inventory. Luckily our function can easily update the table **

In [319]:
insert_columns("inventory", {"wholesale_price": "int", "retail_price": "int"})

In [320]:
get_column_names('inventory')

[i[0] for i in get_column_names('inventory')]

['item_id',
 'item_name',
 'in_stock',
 'total',
 'wholesale_price',
 'retail_price']

In [321]:
dummy_inventory = [('GPU', True, 100, 260, 320), ('CPU', True, 120, 180, 260), ('monitor', True, 88, 120, 160),
                  ('Mouse', False, 0, 30, 45)]

for item in dummy_inventory:
    insert_rows("inventory", "(item_name, in_stock, total, wholesale_price, retail_price)", item)

In [322]:
cur.execute("SELECT * FROM inventory")
cur.fetchall()

[(1, 'GPU', True, 100, 260, 320),
 (2, 'CPU', True, 120, 180, 260),
 (3, 'monitor', True, 88, 120, 160),
 (4, 'Mouse', False, 0, 30, 45)]

# Normal Form

**Normalization is the process of minimizing redundancy from a relation or set of relations. Redundancy in relation may cause insertion, deletion and updation anomalies. So, it helps to minimize the redundancy in relations. Normal forms are used to eliminate or reduce redundancy in database tables.**

** 1. First Normal Form **

If a relation contain composite or multi-valued attribute, it violates first normal form or a relation is in first normal form if it does not contain any composite or multi-valued attribute. A relation is in first normal form if every attribute in that relation is singled valued attribute. **

Arrays vs Relations vs Tuples (and First Normal Form)

Arrays bear some relationship to relations and tuples, but all three are different in significant ways which means they are best at solving different problems.

Like tuples, arrays are ordered.  However:

Unlike tuples every array element must have the same data type.  This means that arrays of text can represent tuples of any type. 
Unlike tuples, arrays do not have a fixed number of elements.  Elements can be added without disturbing the basic type.

Like arrays, relations are basically rectangular, and open ended (items can be added or removed from the end).  However:

Arrays are ordered, relations are not.  This means that an array value is a domain, while a relation value is a set or bag of domains (depending on constraints).
All data types in an array must be identical.  Relations do not have this restriction.

What this means basically is that the mere use of an array in a column of a table does not necessarily violate 1NF, if the array is used in a way that suggests that ordinality matters.

For example we could store matrices for matrix arithmetic as numeric[] arrays.  We could store some problems that are transformed into numeric[] matrices into the db as well (for example simultaneous linear equations) and these would not necessarily violate 1NF. 

In [323]:
cur.execute("SELECT * FROM sales")
cur.fetchall()

[(1, 1, 600, 2, 4, ['GPU', 'monitor']),
 (2, 2, 1200, 3, 5, ['GPU', 'monitor', 'CPU'])]

In [324]:
get_column_names("sales")

[('order_id', 'integer', None),
 ('cust_id', 'integer', None),
 ('purchase_total', 'integer', None),
 ('total_items', 'integer', None),
 ('salesman_id', 'integer', None),
 ('description', 'ARRAY', None)]

In [325]:
cur.execute("ALTER TABLE sales \
DROP COLUMN description;")

cur.execute("ALTER TABLE sales \
DROP COLUMN total_items;")

In [326]:
get_column_names("sales")

[('order_id', 'integer', None),
 ('cust_id', 'integer', None),
 ('purchase_total', 'integer', None),
 ('salesman_id', 'integer', None)]

In [327]:
insert_columns('sales', {"item": "varchar"})

In [328]:
get_column_names("sales")

[('order_id', 'integer', None),
 ('cust_id', 'integer', None),
 ('purchase_total', 'integer', None),
 ('salesman_id', 'integer', None),
 ('item', 'character varying', None)]

In [329]:
cur.execute("SELECT * FROM sales")
cur.fetchall()

[(1, 1, 600, 4, None), (2, 2, 1200, 5, None)]

In [330]:
update_row('sales', 'item', "'GPU'", "order_id = 1")
update_row('sales', 'purchase_total', 200, "order_id = 1")

In [331]:
insert_rows('sales', "(cust_id, purchase_total, salesman_id, item)", (1, 300, 4, "monitor"))

In [332]:
#cur.execute("ALTER TABLE sales ALTER COLUMN order_id \
#TYPE INT;")

#get_column_names("sales")

In [333]:
insert_rows('sales', "(cust_id, purchase_total, salesman_id, item)", (1, 300, 4, "monitor"))
cur.execute("SELECT * FROM sales")
cur.fetchall()

[(2, 2, 1200, 5, None),
 (1, 1, 200, 4, 'GPU'),
 (3, 1, 300, 4, 'monitor'),
 (4, 1, 300, 4, 'monitor')]

In [334]:
update_row('sales', 'item', "'GPU'", "order_id = 2")

insert_rows('sales', "(cust_id, purchase_total, salesman_id, item)", (2, 200, 5, "monitor"))
insert_rows('sales', "(cust_id, purchase_total, salesman_id, item)", (2, 300, 5, "CPU"))

In [335]:
cur.execute("SELECT * FROM sales")
cur.fetchall()

[(1, 1, 200, 4, 'GPU'),
 (3, 1, 300, 4, 'monitor'),
 (4, 1, 300, 4, 'monitor'),
 (2, 2, 1200, 5, 'GPU'),
 (5, 2, 200, 5, 'monitor'),
 (6, 2, 300, 5, 'CPU')]

** In order to get to first normal form we will also need to handle our array values in the IT dept as they do not contain any ordinal relationship **

In [336]:
cur.execute("SELECT * FROM it_dept")
cur.fetchall()

[(3, ['Python', 'SQL', 'Javascript'], True)]

In [337]:
cur.execute("ALTER TABLE it_dept \
DROP COLUMN skills;")

In [338]:
insert_columns('it_dept', {"skill": "varchar"})

In [339]:
update_row('it_dept', 'skill', "'Python'", "emp_id = 3")

** Must remove primary key constraint as now we will have duplciate emp_id's **

In [347]:
cur.execute("ALTER TABLE it_dept DROP CONSTRAINT it_dept_pkey")

** Add foreign key constraint that references employee emp_id so we maintain relationship **

In [354]:
cur.execute("ALTER TABLE it_dept ADD CONSTRAINT fk_emp_id FOREIGN KEY (emp_id) references employee(emp_id);")

In [349]:
languages = ['SQL', 'JAVASCRIPT']

for language in languages:
    insert_rows('it_dept', "(emp_id, skill, remote)", (3, language, True))

In [355]:
cur.execute("SELECT * FROM it_dept")
cur.fetchall()

[(3, True, 'Python'),
 (3, True, 'SQL'),
 (3, True, 'JAVASCRIPT'),
 (3, True, 'SQL'),
 (3, True, 'JAVASCRIPT')]

**2. Second Normal Form **

 - Must have reached 1st Normal Form
 - All columns in the table rely on the primary key

In [232]:
get_column_names("employee") 

[('emp_id', 'integer', None),
 ('first_name', 'character varying', None),
 ('last_name', 'character varying', None),
 ('position', 'character varying', None),
 ('start_date', 'date', None),
 ('phone', 'character varying', None),
 ('email', 'character varying', None)]

In [234]:
get_column_names("sales_dept")

[('emp_id', 'integer', None),
 ('total_sales', 'integer', None),
 ('region', 'character varying', None)]

In [231]:
get_column_names("sales") 

[('order_id', 'integer', None),
 ('cust_id', 'integer', None),
 ('purchase_total', 'integer', None),
 ('salesman_id', 'integer', None),
 ('item', 'character varying', None)]

In [235]:
get_column_names("customers")

[('cust_id', 'integer', None),
 ('first_name', 'character varying', None),
 ('last_name', 'character varying', None),
 ('sales_man', 'integer', None),
 ('contact', 'character varying', None)]

In [236]:
get_column_names("inventory")

[('item_id', 'integer', None),
 ('item_name', 'character varying', None),
 ('in_stock', 'boolean', None),
 ('total', 'integer', None),
 ('wholesale_price', 'integer', None),
 ('retail_price', 'integer', None)]

In [238]:
get_column_names('it_dept')

[('emp_id', 'integer', None),
 ('skills', 'ARRAY', None),
 ('remote', 'boolean', None)]

In [257]:
close_connection()

# Resources 

Normalization : https://www.guru99.com/database-normalization.html#10